In [26]:
import warnings
warnings.filterwarnings("ignore")
import socket,time,math,argparse
import numpy as np
from functions import processCentroids_calib,cameraMatrix_cam1,cameraMatrix_cam2,distCoef_cam1,distCoef_cam2
from cv2 import circle,putText,imshow,waitKey,FONT_HERSHEY_SIMPLEX,destroyAllWindows,triangulatePoints,moveWindow,imwrite
from myLib import orderCenterCoord,getPreviousCentroid,estimateFundMatrix_8norm,decomposeEssentialMat,myProjectionPoints,isCollinear,isEqual,swapElements,getSignal
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import CubicSpline
import os


In [27]:
df = np.genfromtxt('data.csv', delimiter=',')
pts1,pts2 = df[:,0:6],df[:,6:12]

In [28]:
def myReshaping(coord,tol=15):
    # get Y and X distances between markers
    centerX, centerY = np.vstack((coord[:,0], coord[:,2], coord[:,4])).T,np.vstack((coord[:,1], coord[:,3], coord[:,5])).T
    distX = np.array(centerX).max(axis=1) - np.array(centerX).min(axis=1)
    if np.all(distX<tol): maxVec,minVec = centerY[:,2],centerY[:,0]
    elif np.any(distX<tol):
        # looking to the arena, behind the cameras (where the AC is directed to the wall)
        # if lowest marker is from left camera, it is the foremost left marker of the right camera
        # if lowest marker is from right camera, it is the foremost right marker of the left camera
        idxY = np.argmax(distX<tol)
        print(distX,idxY)
        lowestMarker = np.argmax(centerY[idxY])
        if not idxY: # ALWAYS USE CAMERA LEFT AS CAMERA 0
            leftMarker = np.argmin(centerX[1])
            if lowestMarker == leftMarker: return False
            return True
        else: 
            rightMarker = np.argmax(centerX[0])
            if lowestMarker == rightMarker: return False
            return True
    else: maxVec,minVec = centerX[:,2],centerX[:,0]
    # get signal between markers extremities
    firstCam,secCam = getSignal(maxVec[0],minVec[0]),getSignal(maxVec[1],minVec[1])
    if firstCam != secCam: return True
    return False

In [29]:
hasPrevious,centroids1,centroids2=False,[0],[0]
os.system('rm -rf pics/*')
for i in range(0,pts1.shape[0]):
    print(i)
    # check if any of the cameras changes the order of the markers
    swap = myReshaping(np.vstack((pts1[i],pts2[i])))
    # swap if necessary
    if swap: 
        #print('swap')
        #print(pts1[i],pts2[i])
        aux = np.copy(pts1[i][0:2])
        pts1[i][0:2] = np.copy(pts1[i][4:6])
        pts1[i][4:6] = aux
    #else: 
        #print('!!!!! NON SWAP !!!!!') 
        #print(pts1[i],pts2[i])
    # add to common database
    if not hasPrevious: centroids1,centroids2 = np.copy(pts1[i].reshape(-1,2)),np.copy(pts2[i].reshape(-1,2))
    else: centroids1,centroids2 = np.vstack((centroids1, pts1[i].reshape(-1,2))),np.vstack((centroids2, pts2[i].reshape(-1,2)))
    hasPrevious = True
    ### VERBOSE
    img1,img2,k = np.ones((540,960,3))*255,np.ones((540,960,3))*255,0
    for k in range(0,3):
        pt1,pt2 = pts1[i].reshape(-1,2)[k],pts2[i].reshape(-1,2)[k]
        center1,center2 = (int(np.round(pt1[0]*16)),int(np.round(pt1[1]*16))),(int(np.round(pt2[0]*16)),int(np.round(pt2[1]*16)))
        circle(img1,center1,10,(255,0,0),5,shift=4)
        circle(img2,center2,10,(255,0,0),5,shift=4)
        putText(img1,str(k),(int(center1[0]/16)-25, int(center1[1]/16)-25),FONT_HERSHEY_SIMPLEX,0.5,(255,0,0),2) 
        putText(img2,str(k),(int(center2[0]/16)-25, int(center2[1]/16)-25),FONT_HERSHEY_SIMPLEX,0.5,(255,0,0),2) 
    imwrite('pics/'+str(i)+'.jpg',np.hstack((img1,img2)))
    waitKey(1)
    ### VERBOSE'''

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27